In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!pip install kaggle

In [ ]:
!kaggle datasets download -d bhavikjikadara/dog-and-cat-classification-dataset

Dataset URL: https://www.kaggle.com/datasets/bhavikjikadara/dog-and-cat-classification-dataset
License(s): apache-2.0


In [ ]:
!unzip dog-and-cat-classification-dataset.zip -d /content/dataset

In [ ]:
import os
print(os.listdir("/content/dataset"))

In [ ]:
data_dir = "/content/dataset/PetImages"

classes = os.listdir(data_dir)
print("Classes:", classes)

for cls in classes:
    count = len(os.listdir(os.path.join(data_dir, cls)))
    print(f"{cls}: {count} images")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

cat_dir = os.path.join(data_dir, "Cat")
dog_dir = os.path.join(data_dir, "Dog")

cat_images = os.listdir(cat_dir)[:5]
dog_images = os.listdir(dog_dir)[:5]

plt.figure(figsize=(10,5))

# Show cats
for i, img_name in enumerate(cat_images):
    img = mpimg.imread(os.path.join(cat_dir, img_name))
    plt.subplot(2,5,i+1)
    plt.imshow(img)
    plt.title("Cat")
    plt.axis("off")

# Show dogs
for i, img_name in enumerate(dog_images):
    img = mpimg.imread(os.path.join(dog_dir, img_name))
    plt.subplot(2,5,i+6)
    plt.imshow(img)
    plt.title("Dog")
    plt.axis("off")

plt.show()


In [ ]:
!pip install split-folders
import splitfolders

input_folder = "/content/dataset/PetImages"
output_folder = "/content/dataset_split"

# Split into train (70%), validation (20%), test (10%)
splitfolders.ratio(input_folder, output=output_folder, seed=42, ratio=(0.7, 0.2, 0.1))

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        '/content/dataset_split/train',
        target_size=(128, 128),
        batch_size=32,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        '/content/dataset_split/val',
        target_size=(128, 128),
        batch_size=32,
        class_mode='binary')

test_gen = test_datagen.flow_from_directory(
    output_folder + "/test",
    target_size=(128,128),
    batch_size=32,
    class_mode='binary',
    shuffle=False   # don’t shuffle, so we can compare predictions later
)



In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.models import Sequential


In [ ]:
cnn = Sequential()

In [ ]:
cnn.add(Conv2D(filters=32,kernel_size=3,activation='relu',input_shape=(128,128,3), padding='same'))
cnn.add(MaxPooling2D(pool_size=2))

cnn.add(Conv2D(filters=64,kernel_size=3,activation='relu', padding='same'))
cnn.add(MaxPooling2D(pool_size=2))

cnn.add(Flatten())

In [ ]:
cnn.add(Dense(128,activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(64,activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(32,activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(16,activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(8,activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(4,activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(2,activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(1,activation='sigmoid'))

In [ ]:
cnn.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
print(train_generator.class_indices)

In [ ]:
history = cnn.fit(
    train_generator,                 # training data
    epochs=10,                 # number of passes through the dataset
    validation_data=validation_generator    # validation data (to monitor performance)
)

#Prediction

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

In [ ]:
imgPath = '/content/dataset_split/test/Dog/10012.jpg'

img = image.load_img(imgPath, target_size=(128,128))
imgArray = image.img_to_array(img)
imgArray = np.expand_dims(imgArray, axis=0)

img_array = imgArray / 255.0

In [ ]:
prediction = cnn.predict(img_array)

# Since it's binary classification: prediction > 0.5 means "dog", else "cat"
if prediction[0][0] > 0.5:
    print("🐶 It's a Dog!")
else:
    print("🐱 It's a Cat!")

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(img)
plt.axis("off")

label = "Dog 🐶" if prediction[0][0] > 0.5 else "Cat 🐱"
plt.title(f"Prediction: {label}")
plt.show()
